# Importing Required Libraries

In [8]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from typing import Literal

# Constants

In [9]:
ZERO_POINT = -48.6 # erg/s/cm^2/Hz
PLANCK_CONSTANT = 6.63e-34 # J.s
VELOCITY_OF_LIGHT = 299792458 # m/s

# Parameters

In [10]:
# Instrument
class MOS_VIS:
    RESOLUTION: int = 5000
    APERTURE_DIAMETER: float = 0.7 # arcsec
    FIBERS_PER_APERTURE: 7
    SPACIAL_SAMPLING: int = 5 # pixels
    SPECTRAL_SAMPLING: int = 5 # pixels
    PIXELS_PER_OBJECT: int = 175

class MOS_NIR:
    RESOLUTION: int = 5000
    APERTURE_DIAMETER: float = 0.6 # arcsec
    FIBERS_PER_APERTURE: 7
    SPACIAL_SAMPLING: int = 3 # pixels
    SPECTRAL_SAMPLING: int = 3 # pixels
    PIXELS_PER_OBJECT: int = 63

class IFU:
    RESOLUTION: int = 5000
    SPAXEL_SIZE: float = 0.15 # arcsec
    SPACIAL_SAMPLING: int = 3 # pixels
    SPECTRAL_SAMPLING: int = 3 # pixels
    SPAXELS_PER_SPAXEL: int = 27

MOS = { 'VIS': MOS_VIS, 'NIR': MOS_NIR }

# Detector
class DETECTOR_VIS:
    DARK_CURRENT: float = 10/3600 # e/s/pixel
    RON = 3 # e/pixel

class DETECTOR_NIR:
    DARK_CURRENT: float = 0.01 # e/s/pixel
    RON = 3 # e/pixel

DETECTORS = { 'VIS': DETECTOR_VIS, 'NIR': DETECTOR_NIR }

# Telescope
ELT_DIAMETER: float = 38.542 # m

# Download dataset in Google Colab

In [11]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# If we're in Colab, we need to download the data folder from GitHub
if IN_COLAB:
    import os
    
    # Check if the data directory already exists or not
    if not 'data' in os.listdir():
        print("Downloading data...")
        os.makedirs("data", exist_ok=True)

        for file in ["Background.csv", "EE.csv", "Throughput.csv"]:
            os.system(f"wget https://github.com/italoseara/MOSAIC-Prototype/raw/main/data/{file} -O {file}")
            print(f"Downloaded {file}")

DATA_DIR = "./data" if IN_COLAB else "../data"

# Initialize Tables

In [12]:
class LookupTable:
    """A class to represent a lookup table for retrieving data from a CSV file."""

    _df: pd.DataFrame
    _col: str
    
    def __init__(self, csv_file: str, col: str, **kwargs) -> None:
        self._df = pd.read_csv(csv_file, **kwargs)
        self._col = col

    def get(self, key: float) -> dict[str, float]:
        closest_index = self._df[self._col].sub(key).abs().idxmin()
        return self._df.loc[closest_index].to_dict()

gt_table = LookupTable(f"{DATA_DIR}/Throughput.csv", "Wavelength LR")
ee_table = LookupTable(f"{DATA_DIR}/EE.csv", "Wavelength")
bg_table = LookupTable(f"{DATA_DIR}/Background.csv", "Wavelength")

# Funcion Definitions

In [13]:
def signal_to_noise_ratio(
    ndit: int, 
    dit: int,
    wavelength: float,
    source: float,
    sky_condition: Literal["Sky ALL", "Sky NO MOON", "Sky ALL+Th", "Sky NO MOON+Th"], 
    instrument: Literal["MOS VIS", "MOS NIR", "IFU"],
) -> float:
    """Calculate the signal-to-noise ratio for a given observation."""

    if instrument == "IFU":
        print("IFU is not supported yet.")
        return

    band = instrument.split(" ")[1]

    # Get the throughput, encircled energy and sky noise for the given wavelength
    gt = gt_table.get(wavelength)[f"{band}-LR"]
    ee = ee_table.get(wavelength * 1000)[f"EE-LR-ZA45 Durham {band}"]
    sky_noise = bg_table.get(wavelength * 1000)[sky_condition]

    print(f"Global throughput: {gt * 100:.2f}%")
    print(f"Encircled energy: {ee * 100:.2f}%")
    print(f"Sky noise: {sky_noise:.2f} e/s/pixel")

    # Calculate the flux
    wavelength_m = wavelength * 1e-6
    elt_area = np.pi * (ELT_DIAMETER * 100 /2)**2
    energy_per_photon = (VELOCITY_OF_LIGHT * PLANCK_CONSTANT) / (wavelength_m) * 1e7

    flux = (
        10**(-0.4 * (source - ZERO_POINT)) / energy_per_photon * 
        VELOCITY_OF_LIGHT / (wavelength_m)**2 * elt_area *
        wavelength_m / MOS[band].RESOLUTION *
        ee * gt
    )
    counts = flux * dit

    print(f"Flux: {flux:.2f} ph/s/DIT")
    print(f"Counts: {counts:.2f} ph/DIT")
    
    # Calculate the background noise flux
    elt_area = np.pi * (ELT_DIAMETER / 2)**2
    aperture_area = np.pi * (MOS[band].APERTURE_DIAMETER / 2)**2

    bg_flux = sky_noise * elt_area * (wavelength / MOS[band].RESOLUTION) * aperture_area * gt
    bg_counts = bg_flux * dit

    print(f"Background flux: {bg_flux:.2f} ph/s/DIT")
    print(f"Background counts: {bg_counts:.2f} ph/DIT")

    snr = counts * np.sqrt(ndit) / np.sqrt(
        counts + bg_counts + DETECTORS[band].RON**2 *
        MOS[band].PIXELS_PER_OBJECT + DETECTORS[band].DARK_CURRENT *
        dit * MOS[band].PIXELS_PER_OBJECT
    )

    print(f"\nS/R: {snr:.2f}")
    return snr

# Interactive User Inputs

In [18]:
style = {'description_width': '150px'}
layout = {'width': '400px'}

ndit_w = widgets.IntSlider(
    value=60, min=1, max=100, step=1,
    description="Number of exposures", 
    style=style, layout=layout)
dit_w = widgets.IntSlider(
    value=300, min=1, max=3600, step=1,
    description="Exposure time (s)", 
    style=style, layout=layout)
wavelength_w = widgets.FloatSlider(
    value=0.45, min=0.3, max=1.9, step=0.05,
    description="Wavelength (um)", 
    style=style, layout=layout)
source_w = widgets.FloatSlider(
    value=21, min=0, max=100, step=1, 
    description="Source magnitude",
    style=style, layout=layout)
sky_w = widgets.Dropdown(
    value="Sky NO MOON+Th",
    options=["Sky ALL", "Sky NO MOON", "Sky ALL+Th", "Sky NO MOON+Th"],
    description="Sky condition",
    style=style, layout=layout)
instrument_w = widgets.Dropdown(
    options=["MOS VIS", "MOS NIR", "IFU"],
    description="Instrument",
    style=style, layout=layout)

widgets.interactive(
    signal_to_noise_ratio,
    ndit=ndit_w, dit=dit_w,
    wavelength=wavelength_w,
    source=source_w,
    sky_condition=sky_w,
    instrument=instrument_w)

interactive(children=(IntSlider(value=60, description='Number of exposures', layout=Layout(width='400px'), min…